In [376]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [377]:
df =  spark.read.json('people.json')

In [378]:
df.show()


+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# Type of variables
df.printSchema()  prints the types, but sometimes you need to set the correct type for each column.<BR>
For instance, we want to change age type from long to integer.

In [379]:
df.printSchema()



root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [380]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType,StructType)

In [381]:
data_schema = [StructField('age',IntegerType(), True),
               StructField('name',StringType(),True)]  
# True: whether or not the filed can be NULL 
#      (i.e., missing values are allowed)

final_struct = StructType(fields = data_schema)

In [382]:
df =  spark.read.json('people.json',schema=final_struct)

In [383]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



<h1>Select vs grab a data</h1>

<h2>Check column</h2>

**df['age']** returns a **column object**, if you want to actually return a DataFrame you need to use **select**

In [384]:
df['age']

Column<b'age'>

In [385]:
df.select('age')

DataFrame[age: int]

In [386]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [387]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [388]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<h2>Check Row</h2>

In [389]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [390]:
df.head(2)[0]

Row(age=None, name='Michael')

In [391]:
type(df.head(2)[0])

pyspark.sql.types.Row

<h3>To select a specific row from a column of spark data frame:</h3>
If your dataset is small (can fit into memory of driver), you can use **df.collect()[n]**

For example: select row 1 of column "name":

In [392]:
df.select("name").collect()[1]

Row(name='Andy')

## Add/rename a column

In [393]:
df.withColumn('double_age',df['age']**2)  # No inplace change

DataFrame[age: int, name: string, double_age: double]

In [394]:
df.withColumnRenamed('age','new_age').show()# No inplace change

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



# SQL query support in pyspark
First, register the DataFrame as a SQL view. Then, you can run a SQL command. Here, we are naming the SQL view: 'people'.


In [395]:
df.createOrReplaceTempView('people')

In [396]:
results = spark.sql("SELECT * FROM people")

In [397]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [398]:
new_result = spark.sql("SELECT * FROM people WHERE age=30")

In [399]:
new_result.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Filtering

Using **df.filter()**

In [400]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True )
# without inferSchema=True all column types will be considered as string

In [401]:
df.show()


+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [402]:
df.printSchema()


root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



There are two ways for filtering. One using SQL, another way is using python capability.<BR>

Way1: using SQL<BR>
df.filter('close<500').show()    
df.filter('close<500').select(['open','close']).show()

Way2: using python command

In [403]:
df.filter(df['close']<500).select(['open','close']).show()

+------------------+------------------+
|              open|             close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



<h3>Multiple conditions:</h3>

Example:  'close' less than 200 and 'open' larger than 200 <BR>

NB. <BR>
Use **&** instead of **and** <BR>
Do not forget to separate conditions by **parentheses**
    


In [404]:
df.filter((df['close']<200)&(df['open']>200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



## collect() and save the result instead of show():
collect() returns the result as **list** of Rows

In [405]:
result = df.filter((df['close']<200)&(df['open']>200)).collect()

In [406]:
type(result)

list

In [407]:
result

[Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401),
 Row(Date='2010-01-28', Open=204.930004, High=205.500004, Low=198.699995, Close=199.289995, Volume=293375600, Adj Close=25.819922000000002),
 Row(Date='2010-01-29', Open=201.079996, High=202.199995, Low=190.250002, Close=192.060003, Volume=311488100, Adj Close=24.883208)]

In [408]:
row = result[0]

<h3>Rows have useful functions:</h3>

In [409]:
row.asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [410]:
row.asDict()['Volume']

220441900

## Groupby and aggregate functions

<h3>Apply a function on groups: </h3>

In [411]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [412]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [413]:
df.groupby('company')

You can run some functions like mean, min, max, count etc. <BR>
NB. They return a DataFrame

In [414]:
df.groupby('company').mean()

DataFrame[company: string, avg(Sales): double]

In [415]:
df.groupby('company').mean().show()

+-------+-----------------+
|company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



<h3>Aggregate through all rows (group is not IMP for us): </h3>

using **.agg()** and pass a **dictionary**

In [416]:
df.agg({'sales':'sum'}).show()

+----------+
|sum(sales)|
+----------+
|    4327.0|
+----------+



<h3> Apply .agg() on groupby : </h3>

In [417]:
group_data = df.groupBy('company')

In [418]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



<h1>Spark functions</h1>

The way the functions are called is a bit complex!

In [419]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [420]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [421]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [422]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



You can set an alias to the column name:

In [423]:
df.select(stddev('Sales').alias('Std')).show()

+------------------+
|               Std|
+------------------+
|250.08742410799007|
+------------------+



You can format number and name:

In [424]:
from pyspark.sql.functions import format_number

In [425]:
sales_std = df.select(stddev('Sales').alias('Std'))

In [426]:
sales_std.select(format_number('Std',2).alias('Stddev')).show()

+------+
|Stddev|
+------+
|250.09|
+------+



<h2>Order the DataFrame:</h2>

In [427]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



Descending order is a bit complex:

In [428]:
df.orderBy(df['sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Handle Missing data in spark

In [429]:
df = spark.read.csv('ContainsNull.csv', header=True, inferSchema=True)

In [430]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



<h2>Drop:</h2>

In [431]:
df.dropna().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



df.na.drop the same as above function. 

df.na.drop can get **thresh** arg (maximum number of na fileds are necessary to drop a row)

df.na.drop can get **how** arg (any: drop the row if any field is na/all: drop the row if all fields are na)

df.na.drop can get **subset** arg

In [432]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [433]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [434]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



<h2>Filling values:</h2>
NB: fill consideres the type of the values, df.na.fill('missing') only replaces the string type fields.

In [435]:
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



<h3>Filling missing value by mean:</h3>

In [436]:
from pyspark.sql.functions import mean

In [437]:
avg = df.select(mean(df['Sales'])).collect()[0][0]

In [438]:
avg

400.5

In [439]:
df.na.fill(avg , subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [440]:
df.na.fill(avg , ['Sales']).show()  # No need to mention subset= if the order is correct

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# Dates and timestamps

In [441]:
df = spark.read.csv('appl_stock.csv', header=True, inferSchema=True)

In [442]:
df.select('Date','open','close').show()


+----------+------------------+------------------+
|      Date|              open|             close|
+----------+------------------+------------------+
|2010-01-04|        213.429998|        214.009998|
|2010-01-05|        214.599998|        214.379993|
|2010-01-06|        214.379993|        210.969995|
|2010-01-07|            211.75|            210.58|
|2010-01-08|        210.299994|211.98000499999998|
|2010-01-11|212.79999700000002|210.11000299999998|
|2010-01-12|209.18999499999998|        207.720001|
|2010-01-13|        207.870005|        210.650002|
|2010-01-14|210.11000299999998|            209.43|
|2010-01-15|210.92999500000002|            205.93|
|2010-01-19|        208.330002|        215.039995|
|2010-01-20|        214.910006|            211.73|
|2010-01-21|        212.079994|        208.069996|
|2010-01-22|206.78000600000001|            197.75|
|2010-01-25|202.51000200000001|        203.070002|
|2010-01-26|205.95000100000001|        205.940001|
|2010-01-27|        206.849995|

In [443]:
from pyspark.sql.functions import (dayofmonth,hour,
                                   dayofyear,month,
                                   year,weekofyear,
                                   format_number,date_format,
                                   to_timestamp)


<h3>Convert 'Date' from string to datetime: </h3>

using **to_timestamp()**

In [444]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [445]:
df.select(to_timestamp(df['Date'],'yyyy-MM-dd')).show()

+----------------------------------+
|to_timestamp(`Date`, 'yyyy-MM-dd')|
+----------------------------------+
|               2010-01-04 00:00:00|
|               2010-01-05 00:00:00|
|               2010-01-06 00:00:00|
|               2010-01-07 00:00:00|
|               2010-01-08 00:00:00|
|               2010-01-11 00:00:00|
|               2010-01-12 00:00:00|
|               2010-01-13 00:00:00|
|               2010-01-14 00:00:00|
|               2010-01-15 00:00:00|
|               2010-01-19 00:00:00|
|               2010-01-20 00:00:00|
|               2010-01-21 00:00:00|
|               2010-01-22 00:00:00|
|               2010-01-25 00:00:00|
|               2010-01-26 00:00:00|
|               2010-01-27 00:00:00|
|               2010-01-28 00:00:00|
|               2010-01-29 00:00:00|
|               2010-02-01 00:00:00|
+----------------------------------+
only showing top 20 rows



In [446]:
df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



Example: the average closing price per year

In [447]:
from pyspark.sql.functions import  year, countDistinct,mean

In [448]:
result = df.withColumn('year',year(df['Date'])).groupby('year').agg({'close':'mean'})


In [449]:
result.show()

+----+------------------+
|year|        avg(close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



change format_number and alias:

In [450]:
resultTmp = result.withColumnRenamed('avg(close)','average closing price')

In [451]:
finalResult = resultTmp.select(format_number(resultTmp['average closing price'],2).alias('avg closing price'))

In [452]:
finalResult.show()

+-----------------+
|avg closing price|
+-----------------+
|           120.04|
|           472.63|
|           295.40|
|           576.05|
|           104.60|
|           259.84|
|           364.00|
+-----------------+



## Cast column type e.g., from 'string' to 'float'

using **.cast**

In [453]:
finalResult.select(finalResult['avg closing price'].cast('int')).show()

+-----------------+
|avg closing price|
+-----------------+
|              120|
|              472|
|              295|
|              576|
|              104|
|              259|
|              364|
+-----------------+



Check bonus question in DataFramesProjectExercise.ipynb for another example

## The Pearson correlation between High and Volume:


In [454]:
from pyspark.sql.functions import corr
df.select(corr(df['high'],df['volume'])).show()

+------------------+
|corr(high, volume)|
+------------------+
|0.4144024424820674|
+------------------+



## NB: Select actually creates a new dataFrame, 

so you can define as many columns as you want, apply a different function on each column, and set an alias for each column

In [455]:
df.select(year(df['Date']).alias('year'), df['high'].cast('int').alias('high_int'), df['volume']).show()

+----+--------+---------+
|year|high_int|   volume|
+----+--------+---------+
|2010|     214|123432400|
|2010|     215|150476200|
|2010|     215|138040000|
|2010|     212|119282800|
|2010|     212|111902700|
|2010|     213|115557400|
|2010|     209|148614900|
|2010|     210|151473000|
|2010|     210|108223500|
|2010|     211|148516900|
|2010|     215|182501900|
|2010|     215|153038200|
|2010|     213|152038600|
|2010|     207|220441900|
|2010|     204|266424900|
|2010|     213|466777500|
|2010|     210|430642100|
|2010|     205|293375600|
|2010|     202|311488100|
|2010|     196|187469100|
+----+--------+---------+
only showing top 20 rows



# Join DataFrames

<a href="http://www.learnbymarketing.com/1100/pyspark-joins-by-example/#:~:text=Summary%3A%20Pyspark%20DataFrames%20have%20a,join(df2%2C%20df1."> Link </a> 